**Introduction**

This is a Google Colab notebook in which you can test our deep learning model that predicts right ventricular ejection fraction (RVEF) from 2D apical 4-chamber view echocardiographic videos. For detailed information about our model and the entire data analysis pipeline, please refer to our paper (under review) and GitHub repository (https://github.com/rvenet/RVENet-Demo).

This notebook was created to provide a convenient option for testing our model on a few DICOM files. However, if you want to analyze a larger set of DICOM files or modify our code substantially, we recommend running our model on your computer (see the above-referenced GitHub repository).

This notebook consists of text cells (similar to this one) and code cells (see below). Code cells must be run sequentially (by pressing the play button in the top left corner of each code cell).

**Step 0 - Set runtime type**

Google Colab is a hosted Jupyter notebook service that provides access free of charge to computing resources, including GPUs. To run our code with the GPU enabled, go to *Runtime/Change runtime type* and select *GPU* as *Hardware accelerator*. This is a crucial step as our code will throw errors if you do not select *GPU* as the *Hardware accelerator*!

**Step 1 - Install and import required Python packages**

Run the following code cell to install and import all required Python packages.

In [ ]:
!pip install pydicom
#!pip install planar

import os
import numpy as np
import pandas as pd
from datetime import datetime
import pydicom
import torch
from torchvision import models

**Step 2 - Clone the GitHub repository and import the preprocessing code**

Run the following code cell to clone the RVENet-Demo GitHub repository into the Colab environment and import the function required for preprocessing.

In [ ]:
!git clone https://github.com/GerhardHarmsen/RVENet-Demo-no-Planar.git
%cd RVENet-Demo

from preprocessing import get_preprocessed_frames

**Step 3 - Upload your DICOM files - OPTIONAL**

Although we provided an example DICOM file that you may use to test our model, you can also analyze your own DICOM file(s). First, run this code cell to create a new folder (` \content\dicom_files\`) for the DICOM files. Then, use the file-explorer pane (which can be opened by clicking on the folder icon on the left sidebar) to upload your files to the created folder.

In [ ]:
# Creating a folder to which users can upload their DICOM files
dicom_folder_path = "/content/dicom_files/"
os.makedirs(dicom_folder_path, exist_ok=True)

**Step 4 - Set required and optional parameters**

In this cell, you can set all required and optional parameters that will be used in the next steps of the analysis.

In [ ]:
# Set device - DO NOT CHANGE!
device = torch.device("cuda:0")
# NOTE: To run our model, a CUDA-enabled GPU is required!

# Set URL to model weights - DO NOT CHANGE!
model_url = "https://www.dropbox.com/s/d1w0nh1rzclo4ox/full_ensemble_model.pt?dl=1"

# Set URL to example DICOM file - DO NOT CHANGE!
example_dicom_url = "https://www.dropbox.com/s/eqj3uhe1ckijn0y/sample1.dcm?dl=1"

# Path of the folder containing the DICOM files - DO NOT CHANGE!
dicom_folder_path = "/content/dicom_files/"
os.makedirs(dicom_folder_path, exist_ok=True)

# Frame rate of the DICOM files (None or a dictionary) - OPTIONAL
fps_dict = None
# NOTE: If it is set to None (default), the code tries to extract the frame rate from the DICOM tags.
# If the frame rate is not available in the DICOM metadata, 
# the frame rate must be provided for each DICOM file in a dictionary,
# in which the keys are the paths of DICOM files and 
# the values are the corresponding frame rate values as integers
# (e.g., {"/content/dicom_files/sample1.dcm": 60}).

# Heart rate of the patients (None or a dictionary) - OPTIONAL
hr_dict = None
# NOTE: If it is set to None (default), the code tries to extract the heart rate from the DICOM tags.
# If the heart rate is not available in the DICOM metadata, 
# the heart rate must be provided for each DICOM file in a dictionary,
# in which the keys are the paths of DICOM files and 
# the values are the corresponding heart rate values as integers
# (e.g., {"/content/dicom_files/sample1.dcm": 71}).

# Orientation of the left and right ventricles ("Stanford" or "Mayo") - OPTIONAL
orientation = "Mayo"
# Mayo – the right ventricle on the right and the left ventricle on the left side.
# Stanford – the left ventricle on the right and the right ventricle on the left side.

**Step 5 - Download model weights and the example DICOM file**

By running this code cell, model weights and the example DICOM file will be downloaded to the Colab environment.

In [ ]:
# Downloading model weights
model_path = "/content/model.pt"
if not os.path.exists(model_path):
  !wget {model_url} -O {model_path}

# Downloading the example DICOM file if no DICOM files were uploaded by the user
example_dicom_path = "/content/dicom_files/sample1.dcm"
if not os.listdir(dicom_folder_path):
  !wget {example_dicom_url} -O {example_dicom_path}

**Step 6 - Preprocessing**

By running this code cell, DICOM file(s) will be loaded and preprocessed.

In [ ]:
# Searching for DICOM files in the input folder
dicom_files = []
for dir_path, dir_names, file_names in os.walk(dicom_folder_path):
  exclusion_criteria = ["DICOMDIR"]
  dicom_files += [os.path.join(dir_path, f) for f in file_names 
                  if not any(x in f for x in exclusion_criteria)]

# Loading and preprocessing the data from the DICOM file
preprocessed_dicom_data = []
preprocessed_dicom_files = []
for i, dicom_file_path in enumerate(dicom_files):

  # Updating the analysis progress
  analysis_progress = "(" + str(i + 1) + "/" + str(len(dicom_files)) + ")"

  try:
    # Checking whether heart rate was provided explicitly by the user
    if hr_dict:
      if dicom_file_path in hr_dict:
        hr = hr_dict[dicom_file_path]
    else:
      hr = None

    # Checking whether frame rate was provided explicitly by the user
    if fps_dict:
      if dicom_file_path in fps_dict:
        fps = fps_dict[dicom_file_path]
    else:
      fps = None

    # Preprocessing the DICOM file
    preprocessed_dicom_data.append(get_preprocessed_frames(dicom_file_path, fps, hr, orientation))
    preprocessed_dicom_files.append(dicom_file_path)
    
    # Printing analysis progress
    msg_list = [datetime.now().strftime("%m-%d-%Y %H:%M:%S"),
                analysis_progress + " " + dicom_file_path,
                "Preprocessing was successful!"]
    print(" - ".join(msg_list))  
  except pydicom.filereader.InvalidDicomError:
    # Printing error message
    msg_list = [datetime.now().strftime("%m-%d-%Y %H:%M:%S"),
                analysis_progress + " " + dicom_file_path,
                "Invalid DICOM file!"]
    print(" - ".join(msg_list))
    continue
  except Exception as err:
    # Printing error message
    msg_list = [datetime.now().strftime("%m-%d-%Y %H:%M:%S"),
                analysis_progress + " " + dicom_file_path, str(err)]
    print(" - ".join(msg_list))
    continue

**Step 5: Predict RVEF**

As the final step, you can run the model on the preprocessed video(s) to predict RVEF for each.

In [ ]:
# Loading the trained model and sending it to the GPU
model = torch.load(model_path, map_location=device)
model.eval()
model = model.to(device)

# Predicting RVEF for each preprocessed video
list_of_predicted_rvefs = []
list_of_dicom_files = []
for i, preprocessed_frames in enumerate(preprocessed_dicom_data):

  # Updating the analysis progress
  analysis_progress = "(" + str(i + 1) + "/" + str(len(dicom_files)) + ")"

  try:
    # Sending preprocessed frames to the GPU
    preprocessed_frames = preprocessed_frames.to(device, dtype=torch.float)

    # Predicting RVEF for each cardiac cycle in the given video
    cardiac_cycle_predictions = []
    for preprocessed_frames_from_a_single_cardiac_cycle in preprocessed_frames:
      predicted_rvef_from_one_cardiac_cycle = model(preprocessed_frames_from_a_single_cardiac_cycle)
      cardiac_cycle_predictions.append(predicted_rvef_from_one_cardiac_cycle.item())

    # Calculating the final predicted RVEF value for the video (i.e., the mean of cardiac cycle predictions)
    predicted_rvef = np.mean(cardiac_cycle_predictions)

    # Appending the DICOM file path and the predicted RVEF to the lists
    list_of_dicom_files.append(preprocessed_dicom_files[i])
    list_of_predicted_rvefs.append(predicted_rvef)

    # Printing analysis progress
    msg_list = [datetime.now().strftime("%m-%d-%Y %H:%M:%S"),
                analysis_progress + " " + preprocessed_dicom_files[i],
                "Predicted RVEF: {:.3f}%".format(predicted_rvef)]
    print(" - ".join(msg_list))
  except Exception:
    # Printing error message
    msg_list = [datetime.now().strftime("%m-%d-%Y %H:%M:%S"),
                analysis_progress + " " + preprocessed_dicom_files[i],
                "Error in predicting RVEF!"]
    print(" - ".join(msg_list))
    continue

# Saving prediction results to a CSV file
df = pd.DataFrame({"dicom_file_path": list_of_dicom_files, "predicted_rvef": list_of_predicted_rvefs})
df.to_csv("/content/predictions.csv", index=False)